# Manifold learning of images of buildings to identify a trend in architecture

Interdisciplinary project at Technical University of Munihch (TUM CIT, MSc. Computer Science)

@Chair of Urban Development, TUM

## Selected 21 architectural projects
![21ProjectLists](./images/OriginalImage_sample21Images-3.png)
- observation period: 1999-2019
- chose one project per each awardee (in total, 21 architectural projects)
- We chose twenty-one architectural projects, primarily cultural institutions such as auditoriums, conference centres, museums, and schools, designed by Pritzker Prize laureates between 1999 and 2019.


![SampleImages](./images/table21Projects-2.png)

## System Overview
1. Loading dataset (images with additional information)
    - Image scraper
    - Image loader
2. Image pre-processing
    - Image transformer
    - Difference of Gaussians creater
3. Manifold learning
    - Autoencoder
4. Dimensionality reduction
    - LDA dimensionality reduction
5. Visualization of results
    - Visualizer of a latent structure mapping

![SystemOverView](./images/SystemOverview.png)

## Enviroment
- Python 3.10.9
    - Requirements: pip-install requirement.txt
- Manjaro Linux
- Anaconda 22.9.0
    - Python 3.10.9
    - Package list (check ./requirements.txt)
- Jupyter Notebook

# List of 21 architectural projects

In [ ]:
ListOf21ap = ["Carré d'Art", "Educatorium", "Tate Modern", "Arthur and Yvonne Boyd Education Center", "Sydney Opera House", "Contemporary Arts Center", "Diamond Ranch High School", "São Paulo Museum of Art", "Centre Pompidou", "Guthrie Theater", "Kunsthaus Bregenz", "New Museum", "Casa das Histórias Paula Rego", "Ningbo History Museum", "Toyo Ito Museum of Architecture, Imabari", "Centre Pompidou-Metz", "Munich Olympic Park", "Siamese Towers", "Musée Soulages", "Tagore Memorial Hall", "Qatar National Convention"]

DictOf21ap = {}
start_year = 1999

for i in range(len(ListOf21ap)):
    DictOf21ap.update({i+start_year : ""})

print("The year of award: The name of building\n")
for index, key in enumerate(DictOf21ap.keys()):
    DictOf21ap[key] = ListOf21ap[index]
    print(f'{key}: {DictOf21ap[key]}')

# Image scraper
- Scrape images from Microsoft Bing and save in a folder

In [ ]:
from utils.Image_scraper import Building
from utils.Image_scraper import Bing_image_Scraper
from utils.Image_scraper import Instant_BingImageScraper
import matplotlib.pyplot as plt
from utils.Archtecture_dataloader import ImageDataset, Memory_ImageDataset
from utils.pickel_io import dump_pckl, load_from_memory
from utils.transformers import GrayScaler, RescaleTransform, compute_image_mean_and_std, NormalizeTransform, ComposeTransform, Center_crop_by_small_axis
import os
import numpy as np
import torchvision.transforms as transforms
from tqdm import tqdm

### Details
- Scrape images from Microsoft Bing with keywords in the above ListOf21ap (The name of buildings)
- Transform the image into 256x256 pixels by using torch vision
- Save them to the respective folder

In [ ]:
# Image transformer
resizer = transforms.Resize(size = 256)
transformer = ComposeTransform([transforms.ToTensor(), Center_crop_by_small_axis, resizer, transforms.ToPILImage()])

# path to directory to save transformed images
path_to_directory = os.path.join(os.getcwd(), 'images', '21projects_selected')

if not os.path.exists(path_to_directory):
    os.mkdir(path_to_directory)

index = 0

if os.path.exists(path_to_directory):
    print(f"You have already scraped images from Microsoft Bing, \ncheck {path_to_directory}")
else:
    for bname in tqdm(ListOf21ap, position=0, leave=True):
        # print(f"-----------{bname}-------------")
        # print(f"1: Scraping images of {bname}...")
        _set = Instant_BingImageScraper(b_name=bname, num_images=100) #The maximum number of images we can scrape from Microsoft Bing is 150 at once
        # print(f"1: Done.")
        
        # print(f"Transforming images...")
        _transformed_images = []
        for im in _set.images:
            _im = transformer(im)
            _transformed_images.append(_im)
        # print(f"Done.")

        path_to_folder = os.path.join(path_to_directory, str(index))
        # print(f"Saving images in to the {path_to_folder}...")
        if not os.path.exists(path_to_folder):
            os.mkdir(path_to_folder)
        for i, pil_im in enumerate(_transformed_images):
            _im_name = str(i) + ".jpg"
            pil_im.save(os.path.join(path_to_folder, _im_name))
        # print(f"Done.")
        # print(f"------------------------------")
        index = index + 1

# Manual Image retrieval
- Retrieve images so that I have only images that can represent the respective buildings' appearance from the outside Refer the Pritzker Prize Official Website (https://www.pritzkerprize.com/)
- The result of this process could be dounloaded from here (https://drive.google.com/file/d/1YwiuVKi9BFPrZdQq6uu48CD1vVNTvlfy/view?usp=sharing)


# Image loader

In [ ]:
import os
from PIL import Image, ImageOps
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv

from utils.Image_scraper import Building
from utils.Image_scraper import Bing_image_Scraper
from utils.Image_scraper import Flicker_image_Scraper
from utils.Archtecture_dataloader import ImageDataset, Memory_ImageDataset
from utils.pickel_io import dump_pckl, load_from_memory
from utils.transformers import GrayScaler, RescaleTransform, compute_image_mean_and_std, NormalizeTransform, ComposeTransform, Center_crop_by_small_axis

# autoencoder and t_sne
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.decomposition import KernelPCA

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

pd.options.mode.chained_assignment = None
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
path_to_Imfolder = os.path.join(os.getcwd(), 'images', '21projects_selected')
print(path_to_Imfolder)

In [ ]:
def byte2image(self, bi_im):
    """convert byte to image
    Args:
    ----
    bytes_im:  byte
               byte image
    Returns:
    -------
    Image: Image    
    """
    im = Image.open(io.BytesIO(bi_im))
    return im

In [ ]:
dict_images = {}

for i in range(21):
    dict = {i : []}
    dict_images.update(dict)

for i in tqdm(range(21), position = 0, leave = True):
    images = []
    folder_name = str(i)
    path_to_currentFolder = os.path.join(path_to_Imfolder, folder_name,)
    # print(path_to_currentFolder)
    for im in os.listdir(path_to_currentFolder):
        # print(im)
        _im = Image.open(os.path.join(path_to_currentFolder, im))
        _array_im = np.asarray(_im)
        images.append(_array_im)
    dict_images[i]=images
print("Done")

Visualize a sample image

In [ ]:
for im in dict_images[0]:
    plt.imshow(im)

# Num of images for each building

In [ ]:
min = 1e10
for i in range(21):
    print("class:", i+1, len(dict_images[i]))
    if min > len(dict_images[i]):
        min = len(dict_images[i])

print("The number of minimum: ", min)

# Set the number of images

In [ ]:
NofIm = 35

# Dump images in .pckl

In [ ]:
from utils.pickel_io import dump_pckl, load_from_memory

_save_path_to_memory = os.path.join(os.getcwd(), 'memory')
_p_name = 'selected_Numpyimages_of_21buildings.pckl'
if not os.path.exists(os.path.join(_save_path_to_memory, _p_name)):
    dump_pckl(dict_images, _save_path_to_memory, _p_name)

# Load images from .pckl
 - These images are already resized to 256x256 pixels

In [ ]:
_path_to_pckl = os.path.join(os.getcwd(), 'memory')
_p_name_of_pckl = "selected_Numpyimages_of_21buildings.pckl"
_memory_dict_images = load_from_memory(path_to_memory= _path_to_pckl, pickle_fname=_p_name_of_pckl)

# Pytorch

In [ ]:
import torchvision
import torch
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch import nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ExponentialLR

In [ ]:
print(f"pytorch version installed: {torch.__version__}")
print(f"pytroch vision version installed: {torchvision.__version__}")

## Pytorch device

In [ ]:
x = torch.tensor([[1, 2], [3, 4]])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print(f"Original Device:  {x.device}")

tensor = x.to(device)
print(f"Current device: {tensor.device}")

check the cuda availability

In [ ]:
if(torch.cuda.is_available()):
    print("GPU is available")
else:
    raise ValueError('GPU is unavailable, Reboot your machine')

# Dataset creation
- Loading additional data for each image
    - Download .pckl which contains following information from      
    - Download link is [here](https://drive.google.com/file/d/1oUmoFhDsHwFjZ1wdDvi9HkNChxNLU7-e/view?usp=sharing) and save it ./memory
- Contents 
    - The year of award [1999, 2019]
    - The year of inauguration
    - The category of cultural institution
    - The name of building
- Creating complete data set for manifold learning

In [ ]:
_path_to_dataset = os.path.join(os.getcwd(), 'memory')
_pickle_fname = 'ImageDataset_Bing_1805_normalized_squarecrop.pckl'

rescaler = RescaleTransform()
gray_scaler = transforms.Grayscale()
mean = [0.47784522, 0.4758804, 0.47578678]
std = [0.29073316, 0.28931433, 0.2894705]

memory_dataset = Memory_ImageDataset(
    path_to_memory=_path_to_dataset,
    pickle_fname= _pickle_fname,
    transform=ComposeTransform([transforms.ToTensor(), transforms.Resize(256), transforms.Normalize(mean, std), gray_scaler])
)

Sample a item

In [ ]:
sample_item = memory_dataset[0]
item_bname = sample_item['name']
print(item_bname)

In [ ]:
# create list of name

bname2index = {}
counter = 0
for i, data in enumerate(memory_dataset):
    if data['name'] not in bname2index:
        _dict = {data['name']: counter}
        bname2index.update(_dict)
        counter = counter + 1
print(bname2index.keys())

bname_list = {}
category_list = {}
estbyear_list = {}
awardyear_list = {}

for i, data in enumerate(memory_dataset):
    _index = bname2index[data['name']]
    if data['name'] not in bname_list:
        _dbname = {_index: data['name']}
        bname_list.update(_dbname)
    if data['category'] not in category_list:
        _dcategory = {_index: data['category']}
        category_list.update(_dcategory)
    if data['year of establishment'] not in estbyear_list:
        _destbyear = {_index: data['year of establishment']}
        estbyear_list.update(_destbyear)
    if data['award_year'] not in awardyear_list:
        _dawardyear = {_index: data['award_year']}
        awardyear_list.update(_dawardyear)
print(bname_list.items())
print(category_list.items())
print(estbyear_list.items())
print(awardyear_list.items())



In [ ]:
list_building_name = []
for key in bname_list.keys():
    list_building_name.append(bname_list[key])
print(list_building_name)

In [ ]:
num_image_per_class = NofIm

c_images = []
c_labels = []
c_categories = []
c_estbyear = []
c_awardyear = []

for i, bname in enumerate(list_building_name):
    _index = bname2index[bname]
    label = bname_list[_index]
    category = category_list[_index]
    estbyear = estbyear_list[_index]
    awardyear = awardyear_list[_index]
    counter = 0
    index_random_list = np.random.permutation(35)
    for idx, rand_id in enumerate(index_random_list):
        im = _memory_dict_images[_index][rand_id]
        if counter >= num_image_per_class:
            break
        _transformed_im = memory_dataset.transform(im)
        im = np.asarray(_transformed_im)
        c_images.append(im)
        c_labels.append(label)
        c_categories.append(category)
        c_estbyear.append(estbyear)
        c_awardyear.append(awardyear)
        counter = counter + 1

In [ ]:
list_building_name = []
for key in bname_list.keys():
    list_building_name.append(bname_list[key])
print(list_building_name)

In [ ]:
print(_memory_dict_images.keys())

# DoGs creation
- DoG: Difference of Gaussian

In [ ]:
import cv2

example for creation of Gaussian blured image

In [ ]:
plt.imshow(c_images[0].squeeze(), cmap='gray')
plt.show()
print(c_images[0].mean())
dog_sample1 = cv2.GaussianBlur(c_images[0], (11,11), 0)
plt.imshow(dog_sample1.squeeze(), 'gray')
plt.show()

creation of Difference of Gaussians for each images

In [ ]:
dogs = []
for im in c_images:
    im = im
    low_sigma = cv2.GaussianBlur(im, (3,3), 0)
    high_sigma = cv2.GaussianBlur(im, (11,11), 0)
    _dog = low_sigma - high_sigma
    max = _dog.max()
    min = _dog.min()
    _dog = (_dog-min)/(max-min)
    dogs.append(_dog)


visualization of an example of Difference of Gaussians

In [ ]:
plt.imshow(dogs[0].squeeze(), 'gray')
print(dogs[0].max())

# Autoencoder

In [ ]:
# output size of conv2Dtransposed()

def get_ConvTransoutputSize(input_size, padding, kernel_size, stride):
    return (input_size-1)*stride - (2*padding) + (kernel_size-1)+1

Check input and output size

In [ ]:
id_list = []
id_lists = []
id_list_test = []

original_x_train = []
x_train = []
y_train = []
train_year = []
train_estbyear = []
train_categories = []

i = 0
j = 0

original_x_valid = []
x_valid = []
y_valid = []
valid_year = []
valid_estbyear = []
valid_categories = []

original_x_test = []
x_test = []
y_test =[]
test_year = []
test_estbyear = []
test_categories = []

image_dataset = dogs

In [ ]:
train_num_image_per_class = len(image_dataset)
print(train_num_image_per_class)
while(j<len(image_dataset)):
    for s in range(NofIm):
        id_lists.append(j+s)
        if(s<20):
            original_x_train.append(image_dataset[j+s])
            x_train.append(image_dataset[j+s])
            y_train.append(list_building_name[i])
            train_year.append(c_awardyear[j+s])
            train_estbyear.append(c_estbyear[j+s])
            train_categories.append(c_categories[j+s])
        elif(s >= 20 and s < 25):
            original_x_valid.append(c_images[j+s])
            x_valid.append(image_dataset[j+s])
            y_valid.append(list_building_name[i])
            valid_year.append(c_awardyear[j+s])
            valid_estbyear.append(c_estbyear[j+s])
            valid_categories.append(c_categories[j+s])
        elif(s >= 25):
            original_x_test.append(c_images[j+s])
            x_test.append(image_dataset[j+s])
            y_test.append(list_building_name[i])
            test_year.append(c_awardyear[j+s])
            test_estbyear.append(c_estbyear[j+s])
            test_categories.append(c_categories[j+s])
    i = i + 1
    j = NofIm * i

single_class = False

check the number of images

In [ ]:
print("training data set")
print(len(x_train))
print("validation data set")
print(len(x_valid))
print("test data set")
print(len(x_test))

create dataloader for training and validation

In [ ]:
train_dataloader = torch.utils.data.DataLoader(x_train, batch_size = 16, shuffle = True, pin_memory=True, num_workers = 2)
valid_dataloader = torch.utils.data.DataLoader(x_valid, batch_size = 16, shuffle = False, pin_memory=True, num_workers = 2)

visualize sample DoG from training data set

In [ ]:
plt.imshow(x_train[0].squeeze(), 'gray')

check length of respective data loader

In [ ]:
print(train_dataloader.__len__())
print(valid_dataloader.__len__())

# Autoencoder

declaration of convolutional block

In [ ]:
def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=(3, 3), padding=1, bias=False),
        # nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace = True),
        nn.Conv2d(out_channels, out_channels, kernel_size=(3, 3), padding=1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace = True)
    )


## Network architecture
![NetworkArchitecture](./images/NetworkArchitecture_UnetwithInOut.png)

## Layers
![Layers](./images/Layers_network-1.png)

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder1 = double_conv(1, 32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding= 0, dilation=1, ceil_mode=False)
        self.encoder2 = double_conv(32, 64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding= 0, dilation=1, ceil_mode=False)
        self.encoder3 = double_conv(64, 128)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2, padding= 0, dilation=1, ceil_mode=False)
        self.encoder4 = double_conv(128, 256)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2, padding= 0, dilation=1, ceil_mode=False)
        self.bottleneck = double_conv(256, 512)

        self.upconv4 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride =2)
        self.decoder4 = double_conv(512, 256)
        self.upconv3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride = 2)
        self.decoder3 = double_conv(256, 128)
        self.upconv2 = nn.ConvTranspose2d(128, 64, kernel_size= 2, stride = 2)
        self.decoder2 = double_conv(128, 64)
        self.upconv1 = nn.ConvTranspose2d(64, 32, kernel_size = 2, stride = 2)
        self.decoder1 = double_conv(64, 32)
        self.last_conv = nn.Sequential(
            nn.Conv2d(32, 1, 1, padding=0),
            nn.Sigmoid()
        )
        self.weight_skip = 0.001
    
    def forward(self, x):
        conv1 = self.encoder1(x)
        x = self.pool1(conv1)

        conv2 = self.encoder2(x)
        x = self.pool2(conv2)

        conv3 = self.encoder3(x)
        x = self.pool3(conv3)

        conv4 = self.encoder4(x)
        x = self.pool4(conv4)

        x = self.bottleneck(x)

        up1 = self.upconv4(x)
        x = self.decoder4(torch.cat([conv4, up1], 1))

        up2 = self.upconv3(x)
        x = self.decoder3(torch.cat([conv3, up2], 1))

        up3 = self.upconv2(x)
        x = self.decoder2(torch.cat([conv2, up3], 1))

        up4 = self.upconv1(x)
        x = self.decoder1(torch.cat([conv1, up4], 1))

        out = self.last_conv(x)
        return out

define the function for weight initialization

In [ ]:
def initialize_weight(module):
    if isinstance(module, (nn.Linear, nn.Conv2d, nn.ConvTranspose2d)):
        nn.init.xavier_normal_(module.weight)

# Training Autoencoder
configuration

![Config](./images/Configuration.png)

In [ ]:
model = Autoencoder().to(device)
model.apply(initialize_weight)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = ExponentialLR(optimizer, gamma = 0.999)
log_every_batch = 20
max_epochs = 25
avg_train_loss = []
avg_test_loss = []

training and validation at each 20 epochs

In [ ]:
for epoch in range(max_epochs):
    model.train()
    train_loss_trace = []
    for batch, x in enumerate(train_dataloader):
        x = x.to(device)
        # predict
        predict = model(x)
        predict = predict.clamp(0, 1)
        predict[predict!=predict] = 0
        # evaluate reconstruction loss (Binary cross entropy)
        loss = F.mse_loss(predict, x)
        # set 0 to the gradient
        optimizer.zero_grad()
        # calculate gradients by backward propagation
        loss.backward()
        # update parameters
        optimizer.step()
        # loss.detach() => separate a loss from the computational graph, and doesn't require gradient
        # detached tensor == tensor
        # tensor.item() == content
        train_loss_trace.append(loss.detach().item())
        if batch % log_every_batch == 0:
            print('Train: Epoch {}, batch {}, ====> loss {}'.format(epoch, batch, loss))
    
    # if you do not call loss.backward()
    # with torch.no_grad()
    with torch.no_grad():
        model.eval()
        test_loss_trace = []
        for batch, x in enumerate(valid_dataloader):
            x = x.to(device)
            predict = model(x)
            predict = predict.clamp(0, 1)
            predict[predict!=predict] = 0
            loss = F.mse_loss(predict, x)
            test_loss_trace.append(loss.detach().item())
            if batch % log_every_batch == 0:
                print('Test: Epoch {}, batch {}, ====> loss {}'.format(epoch, batch, loss))
    scheduler.step()
    _avg_train_loss = np.mean(train_loss_trace)
    avg_train_loss.append(_avg_train_loss)
    _avg_test_loss = np.mean(test_loss_trace)
    avg_test_loss.append(_avg_test_loss)
    print(f"Epoch {epoch} finished -average train loss {_avg_train_loss}, "
    f"average test loss {_avg_test_loss}")



In [ ]:
train_loss = np.array(avg_train_loss)
test_loss = np.array(avg_test_loss)
fig = plt.figure()
fig.suptitle("Autoencoder")

plt.plot(train_loss, label = 'train')
plt.plot(test_loss, label = 'test')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

# Acquisition of low-dimensional latent feature representation of test data set

In [ ]:
test_dataloader = torch.utils.data.DataLoader(x_test, batch_size = 64, shuffle = False, pin_memory=True, num_workers = 2)

In [ ]:
test_label_dataloader = torch.utils.data.DataLoader(y_test, batch_size = 64, shuffle = False, pin_memory=True, num_workers = 2)

In [ ]:
# number of test images
test_num_image_per_class = int(len(x_test)/21)
print(test_num_image_per_class)

## Obtain low-dimensional latent feature representation for test data set
- Use encoder part of trained autoencoder
- Obtain embeddings from bottleneck layer
- Obtain reconstructed images (make data set for further analysis of the quality of trained model)

In [ ]:
model.eval()
eval_image = []
eval_predict = []

with torch.no_grad():
    latent = []
    for batch, x in enumerate(test_dataloader):
        layer_1 = model.encoder1(x.to(device))
        layer_2 = model.pool1(layer_1)
        layer_3 = model.encoder2(layer_2)
        layer_4 = model.pool2(layer_3)
        layer_5 = model.encoder3(layer_4)
        layer_6 = model.pool3(layer_5)
        layer_7 = model.encoder4(layer_6)
        layer_8 = model.pool4(layer_7)
        layer_9 = model.bottleneck(layer_8)
        latent.append(layer_9.cpu())
        eval_image.append(x)
        predict = model(x.to(device)).cpu()
        eval_predict.append(predict)
    latent = torch.cat(latent)
    eval_image = torch.cat(eval_image)
    eval_predict = torch.cat(eval_predict)


## Visualize/Save reconstruction results
Configure the path to save reconstructed images

In [ ]:
_save_figure = True
# save_figure = False
_path_to_image_folder = ""
import os

ex_name = 'Name_of_experiment'

if _save_figure:
   _path_to_image_folder = os.path.join(os.getcwd(), 'images', 'Results',ex_name)
   if not os.path.exists(_path_to_image_folder):
      os.makedirs(_path_to_image_folder, exist_ok=True)

Create/Save/(Visualize) a image for each input-output pair to visualize next to each other for each input

In [ ]:
for i in range(len(eval_image)):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    _b_name = y_test[i]
    ax1.imshow(eval_image[i].squeeze(), cmap='gray')
    ax1.set_title("Input DoG")
    ax2.imshow(eval_predict[i].squeeze(), cmap='gray')
    ax2.set_title("Reconstructed DoG")
    ax1.axis('off')
    ax2.axis('off')
    if _save_figure:
        b_name = _b_name+str(i%10)+".png"
        path_to_folder = os.path.join(_path_to_image_folder, "Autoencoder_result")
        if not os.path.exists(path_to_folder):
            os.makedirs(path_to_folder, exist_ok=True)
        f_name = os.path.join(path_to_folder, b_name)
        plt.savefig(f_name)
    # plt.show()

Permutation of id list of a pair of latent feature representation (1024 dims) and label
- label is going to be the name of building

In [ ]:
n_vis_samples = len(x_test)
print(n_vis_samples)
indices = np.random.choice(len(latent), n_vis_samples, replace = False)
print(indices)
vis_samples = latent[indices]
latent_y = []
latent_binary = []
latent_year = []
for id in indices:
    latent_y.append(y_test[id])
print(vis_samples.shape)

# Dimensionality reduction and visualization
## Linear discriminant analysis dimensionality reduction
For my research purpose, I employed linear discriminant analysis (LDA) dimensionality reduction as my dimensionality reduction algorithm for further data processing for latent structure visualisation and its interpretation. LDA dimensionality reduction is one of the supervised learning algorithms which is often used for classification tasks in machine learning.

LDA dimensionality reduction can be derived from a generative model of a probabilistic classification modelling algorithm based on Baye's rule. A generative model is a probabilistic model which infers the class of a sample by sampling data from a predicted class distribution. Thus, this prediction can be obtained by posterior probability as follows.

$$\mathbf{P}(y=c|\mathbf{x}, \mu, \Sigma, \Pi) =  \frac{\mathit{P}(\mathbf{x}|y=c)\mathit{P}(y=c|\Pi_{c})}{\sum_{l=1}^{21}\mathit{P(\mathbf{x}|y=l)}\textit{P}(y =l|\Pi_{l})}
= \frac{exp(w_c^T\mathbf{x} + w_{c0})}{\sum_{l=1}^{21}exp(w_l^T\mathbf{x} + w_{l0})}$$

This likelihood $\mathit{P}(\mathbf{x}|y=c)$can be expressed given the mean and covariance of class $\mathit{c}$ as prior knowledge as follows. 

$$\mathbf{P}(x|y=c) = \frac{1}{(2\pi)^{\frac{d}{2}} |\Sigma|^{1/2}}exp(-\frac{1}{2}(x-\mu_c)^T\Sigma^{-1}(x-\mu_c))$$

$$\mu_{c} : \text{mean of c's class distribution} \quad
\Sigma : \text{shared covariance matrix} \quad
\Pi_{c} : \text{prior knowledge of class c}
$$

Where I denote an extracted latent feature from bottleneck of encoder of trained neural network with $\mathbf{x}_i = Encoder(\mathbf{IM}_i)$ that has 1024 dimensions where $\mathbf{IM}_i$ represents original image data with 65536 dimensions, I assign $\mathbf{x}_i$ to the class \textit{c} whose $\mu_c$ is the closest in term of Mahalanobis distance that can be obtained by calculating $(x-\mu_c)^T\Sigma^{-1}(x-\mu_c)$, while also accounting for the class prior probabilities. To this end, I assign $\mathbf{x}_i$ to the class \textit{c} which minimizes the negative log posterior. The negative log posterior is expressed as follows. 

$$-log\mathit{P}(y=c|\mathbf{x}, \mu, \Sigma, \Pi) = -\sigma(\alpha)$$

$$\alpha =  w_c^T\mathbf{x} + w_{c0} + Constant$$

$$w_c = \Sigma^{-1}\mu_{c} \quad w_{c0} = -\frac{1}{2}\mu_c^T\Sigma^{-1}\mu_{c} + log\Pi_c$$

Thus, the probability $\mathbf{P}(y_i = c|\mathbf{x_i})$ which represents how likely $\mathbf{x}_i$ belongs to class \textit{c} could be obtained by using maximum a prior as following.

$$\hat{\mu}, \hat{\Sigma}, \hat{\Pi} = \mathit{argmin} \sum_{i=1}^{N = 210}\sum_{c=1}^{C=21}-log[\mathit{P}(\mathbf{x}_i|\mu_c, \Sigma)\mathit{P}(y_i = c | \Pi)]$$

$$\hat{w_c} = \hat{\Sigma}^{-1}\hat{\mu_{c}} ,\quad \hat{w_{c0}} = -\frac{1}{2}(\hat{\mu_{c}})^T\hat{\Sigma}^{-1}\hat{\mu_{c}}+ log\hat{\Pi_{c}}$$

$$\hat{\alpha} = (\hat{w_c})^T\mathbf{x} + \hat{w_{c0}}$$

$$\mathbf{P}(y_i = c|\mathbf{x_i}) = \sigma(\hat{\alpha})$$

I finally assign x to the class $\hat{y_i}$ by choosing the class c such that it maximizes the posterior probability $\mathbf{P}(y_i = c|\mathbf{x}_i)$. 

$$\hat{y_i} = argmax_c\mathbf{P}(y_i = c|\mathbf{x}_i)$$

Since I can also assume that every cluster has isotropic distribution, namely $\Sigma = \mathbf{I}$, this process is equivalent to assigning $\mathbf{x}_i$ to the closest mean in terms of Euclidean distance. In the L2-norm sense, computing distance between $\mu_{c}$ and $x_{i}$ is equivalent to taking distance after I project them onto affine subspace $\mathcal{H}$ of original space. In other words, if $\mathbf{x}_i$ is close to $\mu_{c}$ in 1024 dimensional space, it will be preserved in the lower dimensional space as well. By choosing my target dimension $\mathcal{L} = 2$ for visualization, I will find mapping function $F(\mathbf{x}_i)$ which can maximize the variance of $\mu_c^{*}$ after mapping from 1024 dimension to 2 dimensions as following.

$$F^{*} = max\frac{1}{C}\sum_{c=1}^{C}(F(\hat{\mu_c}))^2$$

$$\textbf{where} \quad F^{*}(\hat{\mu_c}) = \hat{\mu_c}^{*}$$

Finally, I obtain 2-dimensional intrinsic feature representations for each sample $\mathbf{x}_i$ for $i = \{1,2,...,210\}$ from my test data set as follows.

$$F^{*}(\mathbf{x}_i) = \mathbf{x}_i^{*}$$

Since LDA dimensionality reduction attempts to find directions that can separate different classes in the feature space (latent space) as I mentioned, this approach allows us to conduct semantic understanding among twenty-one architectural projects in terms of architectural designs by providing a meaningful low-dimensional representation of data structure. 


In [ ]:
coordinates_list={}

LDA dimensionality reduction

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
data = vis_samples.reshape(n_vis_samples, -1)
coords_LDA = LinearDiscriminantAnalysis(n_components=2).fit_transform(data, latent_y)
coordinates_list["lda"] = coords_LDA

Create a category-index correspondence

In [ ]:
print(coordinates_list.keys())
category2idx = {}
idx2category = {}
category_list = []
for categ in c_categories:
    if(categ not in category_list):
        category_list.append(categ)

for i, c in enumerate(category_list):
    idx2category.update({i: c})
    category2idx.update({c: i})

Create pandas for latent structure visualization

In [ ]:
# create pandas
import pandas as pd
datasets = {}
dict = {}
index = 0
for key in coordinates_list.keys():
    _x=[]
    _y=[]
    _valid_year=[]
    _estb_year =[]
    _category = []
    _original_image = []
    _dog_image = []
    _label = []
    _indices = []
    for idx, (x, y) in zip(indices, coordinates_list[key]):
        _x.append(x)
        _y.append(y)
        _indices.append(idx)
        _label.append(y_test[idx])
        _original_image.append(original_x_test[idx])
        _dog_image.append(x_test[idx])
        _valid_year.append(int(test_year[idx]))
        _estb_year.append(int(test_estbyear[idx]))
        _category.append(category2idx[test_categories[idx]])
    _dataset = {'id': _indices, 'name': _label,'original_image': _original_image, 'dog_image': _dog_image, 'award_year': _valid_year, 'est_year': _estb_year, 'category': _category, 'x': _x, 'y': _y}
    _pd = pd.DataFrame(_dataset)
    datasets[key]=_pd
    _dict = {'id': _indices, 'name': _label,'original_image': _original_image, 'dog_image': _dog_image, 'award_year': _valid_year, 'est_year': _estb_year, 'category': _category, 'coordinate': coordinates_list[key]}
    dict[key] = _dict
    print(_pd)
    index = index + 1

datasets
data_attribute_label_list = []
for label in _dataset.keys():
    data_attribute_label_list.append(label)
data_attribute_label_list = data_attribute_label_list
print(data_attribute_label_list)

# Visualization of obtained latent feature representation
- Latent structure mapping with labels
    1. Green(Summer): the year of award
    2. Red(Autumn): the year of establish
    3. Blue(Winter): the category of institute

In [ ]:
import matplotlib.pyplot as plt
from mlxtend.plotting import category_scatter

colormap_list = ['summer', 'autumn', 'winter']
data_attribute_label_list=['award_year', 'est_year', 'category']
keys = [key for key in datasets.keys()]
for key in datasets.keys():
    for id, label in enumerate(data_attribute_label_list):
        fig= datasets[key].plot.scatter(x='x', y='y', c=label, colormap=colormap_list[id], title = key)
        if key == 'lda' and label == 'award_year':
            b_name = "LDA_ScatterLatentMap.png"
            f_name = os.path.join(_path_to_image_folder, b_name)
            plt.savefig(f_name)
            

Save the latent feature representations for each input in pickel file 

In [ ]:
path_to_memory = os.path.join(os.getcwd(), 'memory')
p_name = ex_name + '_pandas_datasets_latentcoordinates.pckl'
dump_pckl(datasets, path_to_memory, p_name)

Dump the latent feature representations in .pickl

In [ ]:
path_to_memory = os.path.join(os.getcwd(), 'memory')
p_name = ex_name +'_data_dict.pckl'
dump_pckl(dict, path_to_memory, p_name)

In [ ]:
import os
import matplotlib.patches as mpatches

Load the latent feature representations from .pockl

In [ ]:
_temp_path_to_memeory = os.path.join(os.getcwd(), 'memory')
_p_name = ex_name+'_data_dict.pckl'
_memory_dict = load_from_memory(path_to_memory=_temp_path_to_memeory, pickle_fname=_p_name)

# Visualize data set
- Gray-scale image vs. DoG

In [ ]:
from mpl_toolkits.axes_grid1 import ImageGrid

def latent_space_ALLDatasetVis(data_dict, technique: str, save_figure, path_to_image_folder):
    indices = data_dict['id']
    coordinate = data_dict['coordinate']
    bname_list = data_dict['name']
    images = data_dict['original_image']
    dog_images = data_dict['dog_image']
    _award_year_list = data_dict['award_year']
    # 'original_image': _original_image, 'dog_image':
    # target_images = []
    # target_dogimages = []
    # _path_to_image_folder = os.path.join(path_to_image_folder, str(_award_year))
    # if not os.path.exists(_path_to_image_folder):
    #     os.makedirs(_path_to_image_folder, exist_ok=True)
    target_images = []
    target_dogs = []
    all_images = {}
    all_dogImages = {}

    for i in range(25):
        dict_im = {i: []}
        all_images.update(dict_im)
        dict_dog = {i: []}
        all_dogImages.update(dict_dog)

    counter = 0
    for idx, im in enumerate(images):
        _index = int(_award_year_list[idx]) - 1999
        all_images[_index].append(im)
    
    for idx, dog in enumerate(dog_images):
        _index = int(_award_year_list[idx]) - 1999
        all_dogImages[_index].append(dog)

    image_list = []
    dog_list = []

    for key in all_images.keys():
        for im in all_images[key]:
            image_list.append(im)

    for key in all_dogImages.keys():
        for im in all_dogImages[key]:
            dog_list.append(im)
    

    fig = plt.figure(figsize=(100., 100.))
    columns = 10
    rows = 21
    grid = ImageGrid(fig, 111, nrows_ncols=(rows, columns), axes_pad=0.2,)            
    
    counter = 0    
    for ax, im in zip(grid, dog_list):
        # ax.set_title("image {}".format(counter), fontsize = 50)
        ax.imshow(im.squeeze(), 'gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.axis('off')
    plt.autoscale()
    if save_figure:
        b_name = "Dataset_DOG.png"
        f_name = os.path.join(path_to_image_folder, b_name)
        plt.savefig(f_name)
    # plt.show()


    fig1 = plt.figure(figsize=(100., 100.))
    columns = 10
    rows = 21
    grid1 = ImageGrid(fig1, 111, nrows_ncols=(rows, columns), axes_pad=0.2,)

    for ax, im in zip(grid1, image_list):
        # ax.set_title("image {}".format(counter), fontsize = 50)
        ax.imshow(im.squeeze(), 'gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.axis('off')
    plt.autoscale()
    if save_figure:
        b_name = "Dataset_OriginalImage.png"
        f_name = os.path.join(path_to_image_folder, b_name)
        plt.savefig(f_name)
    # plt.show()


In [ ]:
latent_space_ALLDatasetVis(_memory_dict['lda'], 'lda', save_figure=_save_figure, path_to_image_folder=_path_to_image_folder)

# Visualization of latent structure mapping
Visualize latent structure mapping with a class(building) label

In [ ]:
def latent_space_ALLvisualization_dict(data_dict, technique: str, colors_list, building_year_ordered_list, save_figure, path_to_image_folder, ex_name):
    fig, ax = plt.subplots(figsize=(100,100))
    ax.set_title(technique)

    indices = data_dict['id']
    coordinate = data_dict['coordinate']
    bname_list = data_dict['name']
    images = data_dict['original_image']
    dog_images = data_dict['dog_image']
    _award_year_list = data_dict['award_year']
            
    counter = 0
    for idx, (x, y) in enumerate(coordinate):
        _index = int(_award_year_list[idx]) - 1999
        ax.scatter(x, y, s = 10000, color = colors_list[_index])
        counter = counter + 1
    patches = []
    for i, color in enumerate(colors_list):
        patch = mpatches.Patch(color=color)
        patches.append(patch)

    ax.update_datalim(coordinate)
    ax.autoscale()
    ax.set_title(technique, fontsize = 100)
    plt.legend(patches, building_year_ordered_list, fontsize = 50)
    if save_figure:
        b_name = technique+"_"+ex_name+"_2dscatterMap.png"
        f_name = os.path.join(_path_to_image_folder, b_name)
        plt.savefig(f_name)
    plt.show()

Assign different color to each label

In [ ]:
import matplotlib.colors as mcolors
colors = mcolors.CSS4_COLORS
print(len(colors.keys()))
_color_list = []
for key in colors.keys():
    _color_list.append(colors[key])
    
colors_list = []
cnt = 15
while cnt < 148:
    colors_list.append(_color_list[cnt])
    cnt = cnt + int(148/25)

print(colors_list)
print(len(colors_list))
colors_list = colors_list[:25]
print(len(colors_list))


# Visualization of our latent structure mapping
- Download .pckl for the latent structure mapping that we obtained by this process
        - Download link [here](https://drive.google.com/file/d/12XoA7dlHa_QvZEdo6lxrS5G7bu8ClACT/view?usp=sharing)
- Save it ./memory
- The latent structure mapping that we obtained by this process can be visualized by loading following .pckl in memory folder
        - '1508-2'

In [ ]:
# select specific ex_name
# Following ex_name which contains the latent structure mapping used for my semantic understanding of 21 architectural projects
ex_name = '1508-2' 
_temp_path_to_memeory = os.path.join(os.getcwd(), 'memory')
_p_name = ex_name+'_data_dict.pckl'
_memory_dict = load_from_memory(path_to_memory=_temp_path_to_memeory, pickle_fname=_p_name)

In [ ]:
latent_space_ALLvisualization_dict(_memory_dict['lda'], technique = 'lda', colors_list = colors_list, building_year_ordered_list=list_building_name ,save_figure = _save_figure, path_to_image_folder="", ex_name = ex_name)

# Semantic understanding
## Analysis 1
- Compact Cluster vs. Non-compact Cluster

Our first observation pertained to how the images of each architectural project are distributed in the latent structure mapping. By applying Principal Component Analysis (PCA) to each class, I can obtain the two most variant direction and their variance, which can describe the distribution of plots for each building. From this result, I can find which type each cluster belongs to, a compact cluster or a non-compact cluster. To determine those profiles of a cluster, I need to derive the maximum variance ratio (MVR). MVR can be derived as follows where maximum variance $\sigma_{i, \text{max}}^2$ and minimum variance $\sigma_{i, \text{min}}^2$ of ith building. 

$$ \text{MVR}(i) = \frac{\sigma_{i, \text{max}}^2}{\sigma_{i, \text{max}}^2 + \sigma_{i, \text{min}}^2} (\leqq1.0, \text{in 2D})$$

As the MVR gets close to 0.5, a cluster becomes a compact cluster and it will be close to isotropic Gaussian distribution. On the other hand, as the MVR gets close to 1.0, a cluster becomes a non-compact cluster and it will be an anisotropic Gaussian distribution. By sorting the maximum variance ratio among 21 architectural projects as shown in \autoref{fig:SortedMaxRatio}, I could derive the top 3 compact clusters and the top 3 non-compact clusters in the latent structure mapping. These clusters can be seen with segmentation in \autoref{fig:Top3Compact, fig:Top3Noncompact}.


Get centroid of each class cluster and its variance

In [ ]:
def Get_mean_variance(data_dict, technique: str, colors_list, building_year_ordered_list, save_figure, path_to_image_folder, ex_name):
    indices = data_dict['id']
    coordinate = data_dict['coordinate']
    bname_list = data_dict['name']
    images = data_dict['original_image']
    dog_images = data_dict['dog_image']
    _award_year_list = data_dict['award_year']
            
    _counter = 0
    coords = {}

    class_num = int(len(coordinate)/test_num_image_per_class)

    for i in range(len(building_year_ordered_list)):
        coords.update({i : []})

    for idx, (x, y) in enumerate(coordinate):
        _index = int(_award_year_list[idx]) - 1999
        coords[_index].append([x, y])  

    return coords

Obtain centroid coordinates for 21 architectural projects

In [ ]:
centroid_coords = Get_mean_variance(_memory_dict['lda'], technique = 'lda', colors_list = colors_list, building_year_ordered_list=list_building_name ,save_figure = _save_figure, path_to_image_folder="", ex_name = ex_name)
mean_x ={}
mean_y ={}

for i in range(22):
    mean_x.update({i : 0})
    mean_y.update({i : 0})

for key in centroid_coords.keys():
    if key > 20:
        break
    for i in range(10):
        mean_x[key] = mean_x[key]+centroid_coords[key][i][0]
        mean_y[key] = mean_y[key]+centroid_coords[key][i][1]

for i in range(21):
    mean_x[i] = mean_x[i]/10
    mean_y[i] = mean_y[i]/10


In [ ]:
def plot_means(colors_list, building_year_ordered_list, mean_x, mean_y, techniques = 'lda'):
    fig, ax = plt.subplots(figsize=(100,100))
    ax.set_title(techniques)

    class_num = 21

    for i, name in enumerate(building_year_ordered_list):
        if i > 21:
            break
        ax.scatter(mean_x[i], mean_y[i], s = 10000, color = colors_list[i])
        ax.annotate(i, (mean_x[i], mean_y[i]), fontsize = 50)
    
    patches = []

    for i, color in enumerate(colors_list[:class_num]):
        patch = mpatches.Patch(color=color)
        patches.append(patch)
    
    award_year = 1999
    legend_nameList = []

    for id in range(len(building_year_ordered_list)):
        temp = building_year_ordered_list[id]
        legend_nameList.append(str(id) +". "+ temp)

    ax.autoscale()
    ax.set_title(techniques, fontsize = 100)
    plt.legend(patches[:class_num], legend_nameList[:class_num], fontsize = 75)
    plt.show()

In [ ]:
plot_means(colors_list=colors_list, building_year_ordered_list=list_building_name, mean_x=mean_x, mean_y=mean_y, techniques='lda')

Apply PCA to each class distribution

In [ ]:
# PCA
pca_variance_ratio = []
pca_singular_values = []

for key in centroid_coords.keys():
    if key > 20:
        break
    _array = np.array(centroid_coords[key])
    pca = PCA(n_components=2)
    pca.fit(_array)
    pca_variance_ratio.append(pca.explained_variance_ratio_)
    pca_singular_values.append(pca.singular_values_)

creation of pandas for comaprison of MVR

In [ ]:
import pandas as pd
dict_pca_variance_ratio = {}
dict_singular_values = {}
pca_results = {}
ids = []
max_var = []
min_var = []
max_ratio = []
min_ratio = []
names = []
mx_coords = []
my_coords = []

for id, b_name in enumerate(list_building_name):
    if id > 20:
        break
    ids.append(id)
    names.append(b_name)
    max_var.append(pca_singular_values[id][0])
    min_var.append(pca_singular_values[id][1])
    max_ratio.append(pca_variance_ratio[id][0])
    min_ratio.append(pca_variance_ratio[id][1])

data = {'id': ids, 'name': names, 'max_var': max_var, 'min_var': min_var, 'max_ratio': max_ratio, 'min_ratio': min_ratio}
df = pd.DataFrame(data)
# print(df)

Comparison of maximum variance ratio (MVR)

In [ ]:
sorted_df = df.sort_values(by=['max_ratio'])
print(sorted_df)
sorted_df.plot(kind='bar', x = 'name', y = 'max_ratio', color = 'blue')
plt.ylim([0, 1])
plt.title('Maximum variance ratio (sorted)')
plt.xlabel('buiding name')
plt.ylabel('variance')

### Other comparisons
Comparison of maximum variance

In [ ]:
sorted_df = df.sort_values(by=['max_var'])
print(sorted_df)
sorted_df.plot(kind='bar', x = 'name', y = 'max_var')
plt.ylim([0, 15])
plt.title('Maximum variance (sorted)')
plt.xlabel('buiding name')
plt.ylabel('variance')

Comparison of minimum variance

In [ ]:
sorted_df = df.sort_values(by=['min_var'])
print(sorted_df)
sorted_df.plot(kind='bar', x = 'name', y = 'min_var', color = 'green')
plt.ylim([0, 15])
plt.title('Minimum variance (sorted)')
plt.xlabel('buiding name')
plt.ylabel('variance')

Comparison of minimum variance ratio

In [ ]:
sorted_df = df.sort_values(by=['min_ratio'])
print(sorted_df)
sorted_df.plot(kind='bar', x = 'name', y = 'min_ratio', color = 'red')
plt.ylim([0, 1])
plt.title('Minimum variance ratio (sorted)')
plt.xlabel('buiding name')
plt.ylabel('variance')

In [ ]:
def plot_means_separation(colors_list, building_year_ordered_list, mean_x, mean_y, techniques = 'lda'):
    fig, ax = plt.subplots(figsize=(100,100))
    ax.set_title(techniques)

    class_num = 21

    for i, name in enumerate(building_year_ordered_list):
        if i > 20:
            break
        if i + 1999 > 2009:
            ax.scatter(mean_x[i], mean_y[i], s = 10000, color = 'red')
        elif i+1999 == 2009:
            ax.scatter(mean_x[i], mean_y[i], s = 10000, color = 'yellow')
        else:
            ax.scatter(mean_x[i], mean_y[i], s = 10000, color = 'green')
        ax.annotate(i, (mean_x[i], mean_y[i]), fontsize = 50)


    
    patches = []

    patch = mpatches.Patch(color='red')
    patches.append(patch)
    patch = mpatches.Patch(color='yellow')
    patches.append(patch)
    patch = mpatches.Patch(color='green')
    patches.append(patch)
    
    award_year = 1999
    legend_nameList = []

    legend_nameList.append("before 2009")
    legend_nameList.append("2009")
    legend_nameList.append("after 2019")

    ax.autoscale()
    ax.set_title(techniques, fontsize = 100)
    plt.legend(patches[:class_num], legend_nameList[:class_num], fontsize = 75)
    plt.show()

In [ ]:
plot_means_separation(colors_list=colors_list, building_year_ordered_list=list_building_name, mean_x=mean_x, mean_y=mean_y, techniques='lda')